1. Check CUDA

In [22]:
!/opt/bin/nvidia-smi

Thu Nov 12 21:57:28 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

2. Install nvcc4jupyter (nvcc_plugin) and load the plugin
The code uses the plug in : nvcc_plugin from //github.com/andreinechaev/nvcc4jupyter.git to allow the CUDA C code to be written and executable in Colab. 

In [23]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-ewm7o2g9
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-ewm7o2g9
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp36-none-any.whl size=4307 sha256=94dc8349b201be78bbaea6d6f5153ddeffbf1707d27a54e6915aafb50beb0988
  Stored in directory: /tmp/pip-ephem-wheel-cache-los8db74/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin


In [24]:
%load_ext nvcc_plugin

The nvcc_plugin extension is already loaded. To reload it, use:
  %reload_ext nvcc_plugin


3. Write the cuda code (%%cuda --name matrix.cu )

In [25]:
%%cuda --name matrix_cublas.cu 

#include <stdio.h>
#include <cuda.h>
#include <cublas_v2.h>


#define CUDA_CALL(x) do { if ((x) != cudaSuccess) { \
    printf("Error at %s:%d\n", __FILE__, __LINE__);\
    return EXIT_FAILURE; }} while(0)

#define MAX_VAL 100


cudaError_t checkCuda();

__host__ __device__ void printmat(float* M, int W, int H)
{
    printf("\n");
    for(int y = 0; y < H; ++y)
    {
      for(int x = 0; x < W; ++x)
      {
          printf("%f\t", M[y * W + x]);
      }
      printf("\n");
    }
}



// random generation of a matrix

void randomInit(float* data, int size)
{
    for(int i = 0; i < size; ++i)
    {
        data[i] = rand() / (float)RAND_MAX * MAX_VAL;
    }
}

int matrixMultiply(dim3& dimsA, dim3& dimsB, int N)
{
    // Allocate host and device memory for matrices A, B and C
    unsigned int size_A = dimsA.x * dimsA.y;
    unsigned int mem_size_A = sizeof(float) * size_A;
    float* h_A = NULL;
    CUDA_CALL(cudaHostAlloc(&h_A, mem_size_A, cudaHostAllocDefault));
    float* d_A = NULL;
    CUDA_CALL(cudaMalloc(&d_A, mem_size_A));

    unsigned int size_B = dimsB.x * dimsB.y;
    unsigned int mem_size_B = sizeof(float) * size_B;
    float* h_B = NULL;
    CUDA_CALL(cudaHostAlloc(&h_B, mem_size_B, cudaHostAllocDefault));
    float* d_B = NULL;
    CUDA_CALL(cudaMalloc(&d_B, mem_size_B));

    dim3 dimsC(dimsB.x, dimsA.y, 1);
    unsigned int size_C = dimsC.x * dimsC.y;
    unsigned int mem_size_C = sizeof(float) * size_C;
    float *h_C = NULL;
    CUDA_CALL(cudaHostAlloc(&h_C, mem_size_C, cudaHostAllocDefault));
    float *d_C = NULL;
    CUDA_CALL(cudaMalloc(&d_C, mem_size_C));

    // random fill host matrix B
    srand(42);
    randomInit(h_B, size_B);

    // copy host matrix B to device memory
    CUDA_CALL(cudaMemcpy(d_B, h_B, mem_size_B, cudaMemcpyHostToDevice));

    cudaDeviceSynchronize();

    // Allocating CUDA events that will be used for timing
    cudaEvent_t start, stop;
    CUDA_CALL(cudaEventCreate(&start));
    CUDA_CALL(cudaEventCreate(&stop));

    // Starting
    CUDA_CALL(cudaEventRecord(start, NULL));

    const float alpha = 1.0f;
    const float beta  = 0.0f;
    cublasHandle_t handle;
    CUDA_CALL(cublasCreate(&handle));

    for(int i = 0; i < N; ++i)
    {
        // random fill host matrix A
        randomInit(h_A, size_A);

        // copy host matrix h_A to device memory d_A
        CUDA_CALL(cudaMemcpy(d_A, h_A, mem_size_A, cudaMemcpyHostToDevice));

        // formulating matrix C, C = A * B using cuBlas
        CUDA_CALL(cublasSgemm(handle, CUBLAS_OP_N, CUBLAS_OP_N, dimsB.x, dimsA.y, dimsA.x, &alpha, d_B, dimsB.x, d_A, dimsA.x, &beta, d_C, dimsB.x));

        // copying the results from device to host
        CUDA_CALL(cudaMemcpy(h_C, d_C, mem_size_C, cudaMemcpyDeviceToHost));
    }

    // Recording the event stoppage
    CUDA_CALL(cudaEventRecord(stop, NULL));
    CUDA_CALL(cudaEventSynchronize(stop));

    float msec_TotalMatrixMul = 0.0f;
    CUDA_CALL(cudaEventElapsedTime(&msec_TotalMatrixMul, start, stop));

    // Compute and print the performance of the algorithm
    float msec_MatrixMul = msec_TotalMatrixMul / N;
    double flops_MatrixMul = 2.0 * (double)dimsA.x * (double)dimsA.y * (double)dimsB.x;
    double flopsGiga = (flops_MatrixMul * 1.0e-9f) / (msec_MatrixMul / 1000.0f);
    printf("Performance= %.2f GFlop/s, AVGTime= %.3f msec, TotalTime=%.3f msc \n",
        flopsGiga, msec_MatrixMul, msec_TotalMatrixMul);
 

    // Memory Clean up 
    cudaFreeHost(h_A);
    cudaFreeHost(h_B);
    cudaFreeHost(h_C);
    cudaFree(d_A);
    cudaFree(d_B);
    cudaFree(d_C);

    return EXIT_SUCCESS;
} 

int main(int argc, char** argv)
{
    if(checkCuda() != cudaSuccess)
    {
        return 0;
    }
 
    // condition i) A(500*500), B(500*400), N=100
    printf("Performing condition i: A(500*500), B(500*400), N=100\n");
    int N1 = 100;
    dim3 dimsA1(500, 500, 1);
    dim3 dimsB1(400, 500, 1);
    matrixMultiply(dimsA1, dimsB1, N1);
    printf("\n");
 
    // condition ii) A(50*20), B(20*50), N=5000
    printf("Performing condition ii: A(50*20), B(20*50), N=5000\n");
    int N2 = 5000;
    dim3 dimsA2(20, 50, 1);
    dim3 dimsB2(50, 20, 1);
    matrixMultiply(dimsA2, dimsB2, N2);
    printf("\n");
 
    // condition iii) A(6*4000), B(4000*9), N=1000
    printf("Performing condition iii: A(6*4000), B(4000*9), N=1000\n");
    int N3 = 1000;
    dim3 dimsA3(4000, 6, 1);
    dim3 dimsB3(9, 4000, 1);
    matrixMultiply(dimsA3, dimsB3, N3);
    printf("\n");

    return 0;
}

cudaError_t checkCuda()
{
    printf("Checking CUDA...\n");

    int devID = 0;
    cudaError_t error;
    cudaDeviceProp deviceProp;
    error = cudaGetDevice(&devID);

    if (error != cudaSuccess)
    {
        printf("cudaGetDevice returned error %s (code %d), line(%d)\n", cudaGetErrorString(error), error, __LINE__);
        return error;
    }

    error = cudaGetDeviceProperties(&deviceProp, devID);

    if (deviceProp.computeMode == cudaComputeModeProhibited)
    {
        fprintf(stderr, "Error: device is running in <Compute Mode Prohibited>, no threads can use ::cudaSetDevice().\n");
        exit(EXIT_SUCCESS);
    }

    if (error != cudaSuccess)
    {
        printf("cudaGetDeviceProperties returned error %s (code %d), line(%d)\n", cudaGetErrorString(error), error, __LINE__);
    }
    else
    {
        printf("GPU Device %d: \"%s\" with computation capability %d.%d\n\n", devID, deviceProp.name, deviceProp.major, deviceProp.minor);
    }

    return error;
}

'File written in /content/src/matrix_cublas.cu'

In [26]:
!nvcc -o /content/src/matrix_cublas /content/src/matrix_cublas.cu -lcublas

/content/src/matrix_cublas.cu: In function ‘int matrixMultiply(dim3&, dim3&, int)’:
/content/src/matrix_cublas.cu:86:52: warning: comparison between ‘enum cublasStatus_t’ and ‘enum cudaError’ [-Wenum-compare]
     CUDA_CALL(cublasCreate(&handle));
                                                    ^
/content/src/matrix_cublas.cu:97:160: warning: comparison between ‘enum cublasStatus_t’ and ‘enum cudaError’ [-Wenum-compare]
         CUDA_CALL(cublasSgemm(handle, CUBLAS_OP_N, CUBLAS_OP_N, dimsB.x, dimsA.y, dimsA.x, &alpha, d_B, dimsB.x, d_A, dimsA.x, &beta, d_C, dimsB.x));
                                                                                                                                                                ^


4. Print the Performance

In [27]:
!/content/src/matrix_cublas

Checking CUDA...
GPU Device 0: "Tesla T4" with computation capability 7.5

Performing condition i: A(500*500), B(500*400), N=100
Performance= 23.47 GFlop/s, AVGTime= 8.522 msec, TotalTime=852.180 msc 

Performing condition ii: A(50*20), B(20*50), N=5000
Performance= 1.81 GFlop/s, AVGTime= 0.055 msec, TotalTime=276.462 msc 

Performing condition iii: A(6*4000), B(4000*9), N=1000
Performance= 0.83 GFlop/s, AVGTime= 0.518 msec, TotalTime=518.184 msc 

